# Basic Usage

In [1]:
from IPython.parallel import Client
rc = Client()
rc.ids

[0, 1, 2, 3]

Use all engines by `DirectView`. By this view, it specifies engines to use

In [2]:
dview = rc[:]

Here we define the `is_prime` function as usual

In [3]:
with open('../builtin-cpuheavy/prime_list.txt') as f:
    PRIMES = [int(l) for l in f]

def is_prime(n):
    import math  # import until the function is called
                 # make sure all engines import math
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

### Run in parallel

Use `map_async` or `map_sync` to map function to run in parallel

In [4]:
ar = dview.map_async(is_prime, PRIMES[:8])

In [5]:
ar.wait_interactive() # wait blocks, and wait_interactive provide working status

   4/4 tasks finished after   11 s
done


In [6]:
ar.get()

[True, True, True, True, True, False, True, True]

In [7]:
speedup = ar.serial_time / ar.wall_time
speedup

1.9052240105875689

In [8]:
ar.metadata[:1]

[{'after': [],
  'completed': datetime.datetime(2015, 4, 23, 19, 56, 29, 96360),
  'data': {},
  'engine_id': 0,
  'engine_uuid': '0a258603-b29f-4781-a60d-82d47e62175f',
  'error': None,
  'execute_input': None,
  'execute_result': None,
  'follow': [],
  'msg_id': '1f6ced59-4c14-45f3-a97f-8b0ea29eb6e2',
  'outputs': [],
  'outputs_ready': True,
  'received': datetime.datetime(2015, 4, 23, 19, 56, 29, 99394),
  'started': datetime.datetime(2015, 4, 23, 19, 56, 25, 363204),
  'status': 'ok',
  'stderr': '',
  'stdout': '',
  'submitted': datetime.datetime(2015, 4, 23, 19, 56, 25, 361740)}]

More to checkout <http://ipython.org/ipython-doc/dev/parallel/parallel_multiengine.html#calling-python-functions>

## Import modules remotely

If any modules imported, engines should import them as well. So here use a `dview.sync_import()` context_manager to help this issue. Note that `import numpy as np` will not actually intepreted as `np` module on engines but instead remaining `numpy`.

In [9]:
with dview.sync_imports():
    import math
    import numpy as np

importing math on engine(s)
importing numpy on engine(s)


In [10]:
def find_np():
    np.random.randint(10)

rc[:2].apply_sync(find_np)

CompositeError: one or more exceptions from call to method: find_np
[0:apply]: NameError: name 'np' is not defined
[1:apply]: NameError: name 'np' is not defined

# IPython Parallel Magic

To do so, use `%%px` ipython magic. The `%%px` cell block executes its statements on all engines

In [11]:
%%px
import numpy as np
np.random.randint(6)

Out[0:1]: 3

Out[1:1]: 5

Out[2:1]: 3

Out[3:1]: 4

In [12]:
%%px
# try to run it multiple times, engines use same processes (like a remote Python intepreter)
import os
os.getpid()

Out[0:2]: 27338

Out[1:2]: 27339

Out[2:2]: 27340

Out[3:2]: 27341

## Passing/Collecting Data

Pushing / pulling a variable to all engines

In [13]:
# push
dview['prog'] = 'val_prime'

# pull
dview['prog']

['val_prime', 'val_prime', 'val_prime', 'val_prime']

### Splitting a variable across engines

In [14]:
# all engines get x but with differnt value
ar = dview.scatter('x', list(range(13)))
ar.wait()

In [15]:
dview['x']

[[0, 1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]

In [16]:
# get x from all engines and combined
dview.gather('x', block=True)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

Here is another example

In [17]:
%%px 
import numpy as np
rand_n = np.random.randint(0, 10, 6)

In [18]:
dview['rand_n']

[array([2, 1, 5, 0, 6, 5]),
 array([6, 5, 5, 2, 6, 6]),
 array([0, 4, 7, 6, 7, 3]),
 array([8, 7, 7, 7, 3, 0])]

In [19]:
dview.gather('rand_n', block=True)

array([2, 1, 5, 0, 6, 5, 6, 5, 5, 2, 6, 6, 0, 4, 7, 6, 7, 3, 8, 7, 7, 7, 3,
       0])

In [20]:
# sum at each engine
def rand_sum():
    return np.sum(rand_n)

ar = dview.apply_sync(rand_sum)

In [24]:
ar

[19, 30, 27, 32]

In [25]:
# parallel sum shoud equal to serial sum
sum(ar) == sum(dview.gather('rand_n', block=True))

True